In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
# Double induction
# - over dimension of the opetope
# - over dimension of the faces 

In [41]:
from Opetope import Opetope, Face, flatten, clear_cache, MEMO

In [42]:
# test1 - OK
a = Opetope(name='a')
b = Opetope(name='b')
c = Opetope(name='c')
d = Opetope(name='d')

ab = Opetope(ins=[a], out=b, name='ab')
bc = Opetope(ins=[b], out=c, name='bc')
cd = Opetope(ins=[c], out=d, name='cd')
ad = Opetope(ins=[a], out=d, name='ad')
ac = Opetope(ins=[a], out=c, name='ac')

alpha = Opetope(ins=[ab, bc], out=ac, name='alpha')
beta = Opetope(ins=[ac, cd], out=ad, name='beta')
gamma = Opetope(ins=[ab, bc, cd], out=ad, name='gamma')

whole1 = Opetope(ins=[alpha, beta], out=gamma, name='whole1')
print(whole1)

assert alpha.shape() == beta.shape()
assert alpha.shape() != gamma.shape()

(whole1: [(alpha: [(ab: [a] -> b), (bc: [b] -> c)] -> (ac: [a] -> c)), (beta: [(ac: [a] -> c), (cd: [c] -> d)] -> (ad: [a] -> d))] -> (gamma: [(ab: [a] -> b), (bc: [b] -> c), (cd: [c] -> d)] -> (ad: [a] -> d)))


In [43]:
from collections import Counter

In [47]:
from typing import Set
from pdb import set_trace
from typing import List

def build_possible_opetopes(op, building_blocks, P, Q):
    # build all possible opetopes which have the codomain == op
    # and are constructed only from elems
    
    # and proceed from here by DFS
    results = set()
    print("Starting DFS, out {} blocks {} P {} Q {}".format(op, building_blocks, P, Q))
    
    DFS({op.out}, set(), building_blocks, results, op, P, Q)
    return results
    
def DFS(current_ins: List[Face], used: List[Face], building_blocks: List[Face], results, target_out: Face, P: Opetope, Q: Opetope):
#     set_trace()
    
    # if all "used" faces together form a valid opetope, we win 
    if (Opetope.match(ins=[u.p1 for u in used], out=target_out.p1) and
        Opetope.match(ins=[u.p2 for u in used], out=target_out.p2)):
        results.add(Face(p1=P, p2=Q, ins=used, out=target_out))
#         print("Found face!!!")
        return
    
    # ugly hack, but points do not have themselves as outs, so it is needed
    out = lambda x: x if not x.level else x.out
    
    # if not, we have to iterate through all possible to use opetopes and check each combination recursively
    for b in building_blocks:
        for i in current_ins:
#             print("Now focusing on b: {} u: {}".format(b, i))
            if i.p1 == out(b.p1) and i.p2 == out(b.p2):
                print("Used")
                DFS({*current_ins, *b.ins} - {i}, 
                    {*used, b}, 
                    {x for x in building_blocks if x != b}, 
                    results, 
                    target_out, P, Q)
    
    return
    
# todo change Set[Face] to OpetopicNet, imposing appropriate restrictions
def product(P: Opetope, Q: Opetope, small_faces: Set[Face]) -> Set[Face]:
    
    print("Now analyzing opetopes {} and {}".format(P, Q))
    subs1 = P.all_subopetopes()
    subs2 = Q.all_subopetopes()

    # product is a set of Faces
    # small_faces, because these are the ones that don't map to whole op1 and whole op2 simultaneously
    
    points = lambda s: {p for p in s if not p.level}
    arrows = lambda s: {p for p in s if p.level == 1}
    small_faces |= {Face.from_point_and_point(s1, s2) for s1 in points(subs1) for s2 in points(subs2)}
    small_faces |= {Face.from_arrow_and_point(s1, s2) for s1 in arrows(subs1) for s2 in points(subs2)}
    small_faces |= {Face.from_point_and_arrow(s1, s2) for s1 in points(subs1) for s2 in arrows(subs2)}
    small_faces |= {Face.from_arrow_and_arrow(s1, s2) for s1 in arrows(subs1) for s2 in arrows(subs2)}
    
    
    for s1 in subs1:
        for s2 in subs2:
            # starting induction with just points
            # hopefully, it will work
            # if not, then FIXME start induction with both points and arrows

            if (s1, s2) != (P, Q) and (s1.level, s2.level) >= (1, 1):
                small_faces |= product(s1, s2, small_faces)
    
    # now we only have to construct big_faces the faces which map simultaneously to whole op1 and whole op2
    # minimal dimension of such a face is k = min(dim(P), dim(Q))
    big_faces = set()
    k = min(P.level, Q.level)
    
    # induction on l - dimension of such face
    l = k
    
    # we proceed until there is no new face
    while True:
        # we have constructed all big faces of dimension < l
        l += 1
        
        # the possible codomains of such a face are:
        possible_codomains = []
        # 1) all (l-1)-dimensional big_faces
        possible_codomains += [f for f in big_faces if f.level < l]
        
        if l == k + 1:
            possible_codomains += [f for f in small_faces if f.p1 == P and f.p2 == Q]
        
        # 2) if dim(P) <= dim(Q), then it may be a face that maps to P and codomain(Q)
        if P.level < Q.level:
            possible_codomains += [f for f in small_faces if f.p1 == P
                                                          and f.p2 == Q.out]
        # 3) if dim(Q) <= dim(P), then it may be a face that maps to Q and codomain(P)
        if Q.level < P.level:
            possible_codomains += [f for f in small_faces if f.p1 == P.out
                                                          and f.p2 == Q]
            
        # 4) all faces that map to codomain(P) and codomain(Q) - I DON'T THINK THIS IS CORRECT
#         possible_codomains += [f for f in small_faces if f.p1 == P.out
#                                                       and f.p2 == Q.out]
        
#         set_trace()

        # now, for each possible codomain, we build the opetope that contains it
        new_opetopes = set()
        for f in possible_codomains:
            # TODO it won't work, f doesn't have "level"
            # I think it is enough to build just from the stuff that has the right dimension
            # eg, equal to dim(f)
            building_blocks = {s for s in small_faces if s.level == f.level}
            new_opetopes |= build_possible_opetopes(op=f, building_blocks=building_blocks, P=P, Q=Q)
        
        big_faces |= new_opetopes
        
        if not new_opetopes:
            return small_faces | big_faces
        

In [48]:
op1 = Opetope(ins=[a], out=b, name="ab")
op2 = Opetope(ins=[c], out=d, name="cd")

p = product(op1, op2, set())
p

Now analyzing opetopes (ab: [a] -> b) and (cd: [c] -> d)
Starting DFS, out ((ab: [a] -> b), (cd: [c] -> d))!1
 blocks {((ab: [a] -> b), d)!1
, ((ab: [a] -> b), c)!1
, (a, (cd: [c] -> d))!1
, ((ab: [a] -> b), (cd: [c] -> d))!1
, (b, (cd: [c] -> d))!1
} P (ab: [a] -> b) Q (cd: [c] -> d)
Used
Used
Used
Used
Used
Starting DFS, out ((ab: [a] -> b), (cd: [c] -> d))!2
 blocks set() P (ab: [a] -> b) Q (cd: [c] -> d)


{(b, c)!0,
 (b, d)!0,
 ((ab: [a] -> b), c)!1,
 (a, c)!0,
 (a, d)!0,
 (b, (cd: [c] -> d))!1,
 ((ab: [a] -> b), d)!1,
 ((ab: [a] -> b), (cd: [c] -> d))!2,
 (a, (cd: [c] -> d))!1,
 ((ab: [a] -> b), (cd: [c] -> d))!1}

In [49]:
len(p)

10